In [2]:
import pandas as pd

In [15]:
df_train = pd.read_pickle('../input/5th-place-solution-code/x_train.pkl')
df_train['PetID'] = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')['PetID'].values
df_test = pd.read_pickle('../input/5th-place-solution-code/x_test.pkl')
df_test['PetID'] = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')['PetID'].values


In [16]:
df_all = df_train.append(df_test)

In [17]:
def proc(x):
    try:
        x.astype('float32')
        return True
    except ValueError:
        return False

In [18]:
cols = [col for col in df_all if proc(df_all[col]) and col != 'AdoptionSpeed']

In [19]:
df_all[cols]

,act_0,act_1,act_2,act_3,act_4,act_5,act_6,act_7,act_8,act_9,...,Type_label,Dewormed_Vaccinated_label,Breed2_label,State_label,Gender_label,Color1_label,Color2_label,Color3_label,no_name_label,top_label_description_1_label
0,-0.830160,1.425100,1.706436,-0.262937,-0.993541,-0.970723,-0.544012,3.727692,3.319294,0.117710,...,1,4,0,2,0,0,6,0,0,12
1,-0.862162,-0.393488,1.332435,0.687813,-0.997934,-0.993283,2.058035,-0.941668,-0.698512,-0.569867,...,1,8,0,13,0,0,1,0,1,12
2,0.458738,-0.473508,-0.995439,-0.983077,-0.995737,-0.669550,2.407662,-0.901308,0.112228,-0.973742,...,0,0,0,2,0,1,6,0,0,17
3,-0.604201,-0.849263,-0.985205,-0.977387,-0.993572,-0.922073,3.285267,-0.821653,0.141475,-0.973828,...,0,0,0,13,1,0,1,0,0,17
4,0.672229,-0.744274,-0.959908,-0.895013,-0.988739,-0.878957,3.890141,-0.948924,-0.777626,-0.989966,...,0,4,0,2,0,0,0,0,0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,0.370184,0.763091,-0.988368,-0.987839,-0.994572,-0.635198,-0.834051,-0.940240,-0.749309,-0.990894,...,0,0,144,13,0,0,1,0,0,17
3968,0.268106,-0.779415,-0.916523,-0.947959,-0.992273,-0.685320,-0.606110,-0.873869,1.018224,-0.816076,...,0,2,144,2,0,0,1,5,0,17
3969,-0.990074,-0.172127,5.242781,-0.469064,-0.998022,-0.999760,-0.989876,6.652140,6.722751,-0.178591,...,1,7,0,13,2,0,1,2,1,12
3970,3.733073,3.075843,-0.975182,-0.678539,-0.995656,1.177959,-0.937936,-0.163092,-0.876426,-0.901398,...,0,4,0,1,1,4,6,0,0,17


In [20]:
df_all = df_all[['PetID'] + cols]#df_train.columns.values.tolist()]

In [21]:
import numpy as np
import glob
import joblib
import os
from PIL import Image
import imagehash
from tqdm.auto import tqdm

paths = glob.glob('../input/petfinder-adoption-prediction/*/*jpg')

result = joblib.Parallel(n_jobs=-1, verbose=1)(joblib.delayed(lambda x: imagehash.average_hash(Image.open(x)))(path) for path in paths)


tmp = pd.DataFrame()
tmp['path'] = paths
tmp['hash'] = result
tmp['hash'] = tmp['hash'].astype(str)

tmp['is_train'] = tmp['path'].str.find('train') > -1
tmp['is_test'] = tmp['path'].str.find('test') > -1
tmp['idx'] = tmp['path'].map(lambda x: int(x.split('-')[-1][:-4]))
tmp['PetID'] = tmp['path'].map(lambda x: x.split('/')[-1].split('-')[0])

df1_train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
df1_test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')

aaa = [df1_test['PetID'].map(pd.read_csv(path, index_col='PetID')['AdoptionSpeed']).values
 for path in ['../input/pet-data/prev1st.csv']]
aaa = np.mean([t for t in aaa if not np.isnan(t.sum())], axis=0)

df1_test['AdoptionSpeed'] = aaa
df1_train = df1_train.append(df1_test)

df1 = pd.merge(tmp, df1_train, how='left', on='PetID')

df1['is_train'] = df1['is_train'].astype(np.float32)
df1['is_test'] = df1['is_train'].astype(np.float32)
df1['hash'] = df1['hash'].astype(str)
#df1 = df1[(df1['is_train'] > 0 ) | (~df1['hash'].isin(df1.loc[df1['is_train'] > 0, 'hash']))].reset_index(drop=True)
#df1.to_csv('df1_train_test_v6.csv', index=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1744 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 4544 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 8144 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 12544 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 17744 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 23744 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 33504 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 48704 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 65504 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 72776 out of 72776 | elapsed:   36.5s finished


In [22]:

df_all2 = pd.merge(df1[['PetID', 'AdoptionSpeed', 'hash']], df_all, on='PetID', how='left')

In [23]:
#df_all2.to_csv('../input/pet-data/df1_train_test_all_3.csv', index=False)

In [24]:
df_all2.to_feather('../input/pet-data/df1_train_test_all_4ndplace.ftr')

In [25]:
df_all2[['AdoptionSpeed']].isna().mean()

AdoptionSpeed    0.0
dtype: float64